# AY08 - Iterables, Iteradores y builtins

### Autores:  Caua Paz, ~~Strovelight~~ Pablo Araneda, Julio Huerta

## Built-ins 
### ```__getitem__```, ```enumerate```, ```zip```

Los _built-in_ son funciones que ya vienen implementadas en python. Estas funciones se pueden usar también en clases creadas por tí. Por ejemplo ```__getitem__``` es un _built-in_, solo que usualmente no se conoce por ese nombre

In [ ]:
# probemos el __getitem__ de una funcion
lista_numerica = [2,2,3,3]
print(f"primer elemento de la lista: {lista_numerica[0]}")

El ```__getitem__``` es el que te permite acceder a un elemento de la lista de lista. Veamos ahora si intenramos crear una clase sin eso

In [ ]:
class CalendarioSimple:
    def __init__(self, dia_hoy, mes_hoy):
        self.dia_hoy = dia_hoy
        self.mes_hoy = mes_hoy

In [ ]:
MiCalendario = CalendarioSimple(10,3)
MiCalendario[0]

Como se puede ver, al no tener el ```__getitem__```, el programa se cae. Veamos que pasa si le creamos la función

In [ ]:
class CalendarioDeHoy:
    def __init__(self, dia_hoy, mes_hoy):
        self.dia_hoy = dia_hoy
        self.mes_hoy = mes_hoy
    def __getitem__(self, index):
        return f"El dia es {self.dia_hoy+index}, y el mes es {self.mes_hoy}"

In [ ]:
MiCalendario = CalendarioDeHoy(10,3)
print(f"consulta del elemento 0: {MiCalendario[0]}")
print(f"consulta del elemento 10: {MiCalendario[10]}")
print(f"consulta del elemento -9: {MiCalendario[-9]}")
print(f"consulta del elemento 100: {MiCalendario[100]}")
print(f"consulta del elemento -100: {MiCalendario[-100]}")

Como se puede observar, el calendario funciona parcialmente para consultar días pasados y actuales, pero no cambia de mes. Necesitamos la información de los días del mes para poder mostrar mejor la fecha. Para esta tediosa tarea le pediste a un amigo que te de la información, y te la dio de la siguiente forma:

In [ ]:
meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
dias_mes = [  31,        28,      31,      30,     31,      30,      31,       31,           30,        31,          30,          31]

Lamentablemente, no te sirve mucho la forma en la que tu amigo te dio la información. Sin embargo, aquí es cuando ```zip``` y ```enumerate``` llegan al rescate. Primero veamos ```enumerate```.

## ```enumerate```
```enumerate``` retorna una tupla con un valor que va enumerando (contando crecientemente) a partir del primer elemento. Como usualmente en la forma de escritura de mes, enero es 1, y diciembre 12, queremos partir contando el mes de 1. Sé que esto puede sonar mal para los que cuentan de 0, pero creo que la mayoría al decir mes 3, pensó en marzo. Puede que este caso parezca problemático, pero ```enumerate``` también cubre este tipo de casos:

In [ ]:
numero_primer_mes = 1
for numero_mes, mes in enumerate(meses, numero_primer_mes):
    print(f"El mes numero {numero_mes} es {mes}")

Enumerate funciona tal que el primer parámetro sea el iterable, y el segundo sea con qué valor empezar a contar (por defecto es 0). Ahora con la información anterior, podemos crear diccionarios tal que se pueda acceder al día con facilidad

In [ ]:
diccionario_nombre_mes = dict()
for numero_mes, mes in enumerate(meses, numero_primer_mes):
    diccionario_nombre_mes[numero_mes] = mes
print(f"El mes numero {2} es {diccionario_nombre_mes[2]}")

Ahora podremos mostrar los meses con su nombre. Sin embargo, todavía queda el problema de los días. Aquí es donde podemos usar ```zip``` paca cumplir nuestra misión.

## ```zip```

```zip``` permite juntar el contenido de varios iterables e irlos recorriendo por cada entrada, es decir, se itera el primer elemento de cada iterable primero, luego los segundos, y así sucesivamente.
Hablemos del tan usado ```range()```. Este retorna un iterador, y en conjunto con la lista de los días de cada mes, podemos usar ```zip()``` para juntarlos y que se entreguen las tuplas.

In [ ]:
# aprovechamos con la iteración crear el diccionario de dias de cada mes
diccionario_dias_mes = dict()
for numero_mes, dias_del_mes in zip(range(1,13), dias_mes):
    diccionario_dias_mes[numero_mes] = dias_del_mes
    print(f"El mes numero {numero_mes} tiene {dias_del_mes} dias")

Para las personas observadoras, pueden ver que lo anterior también se puede hacer con ```enumerate()```, y que resulta mas corto:

In [ ]:
for numero_mes, dias_del_mes in enumerate(dias_mes, 1):
    print(f"El mes numero {numero_mes} tiene {dias_del_mes} dias")

Sin embargo, ```zip()``` se encarga de agrupar iterables, y mostrar los elementos de cada uno por orden, por lo que se puede hacer lo siguiente:

In [ ]:
for numero_mes, nombre_mes, dias_del_mes in zip(range(1,13), meses, dias_mes):
    print(f"El mes numero {numero_mes} llamado {nombre_mes} tiene {dias_del_mes} dias")

Ahora solo queda crear el calendario usando los diccionarios de antes

In [ ]:
class Calendario:
    dias_mes = diccionario_dias_mes
    nombre_mes = diccionario_nombre_mes

    def __init__(self, dia_hoy, mes_hoy):
        self.dia_hoy = dia_hoy
        self.mes_hoy = mes_hoy
        print(f"El dia de hoy es {dia_hoy} de {self.nombre_mes[mes_hoy]}")

    def __getitem__(self, index):
        mes_pedido = self.mes_hoy
        dia_pedido = self.dia_hoy + index
        # se decide iterar hasta encontrar el dia correcto
        while not (0 < dia_pedido <= self.dias_mes[mes_pedido]):
            if dia_pedido <= 0:   # se necesita retroceder mes
                mes_pedido -= 1
                # hay que tener cuidado de quedar en un mes inválido
                if mes_pedido == 0:
                    mes_pedido = 12
                dia_pedido += self.dias_mes[mes_pedido]
            elif dia_pedido > self.dias_mes[mes_pedido]: # se necesita un mes futuro
                dia_pedido -= self.dias_mes[mes_pedido]
                mes_pedido += 1
                # hay que tener cuidado de quedar en un mes inválido
                if mes_pedido == 13:
                    mes_pedido = 1
        return f"El dia es {dia_pedido}, y el mes es {self.nombre_mes[mes_pedido]}"

Probemos ahora el código:

In [ ]:
Hoy = Calendario(dia_hoy=3, mes_hoy=10)

In [ ]:
# probemos ahora distintas consulyas
print("Consulta con  '-3':", Hoy[-3])
print("Consulta con  '31':", Hoy[31])
print("Consulta con   '0':", Hoy[0])
print("Consulta con '365':", Hoy[365])

## Iterables e Iteradores

Un **iterable** es cualquier objeto sobre el cual se puede iterar, por lo mismo, cualquier iterable podría aparecer al lado derecho de un for loop (for i in iterable:). Un iterable contiene el método `__iter__()`. Se puede iterar todas las veces que uno quiera sobre un iterable, como en el caso de las listas por ejemplo.

Un **iterador** es un objeto que itera sobre un iterable, es el objeto retornado por el método `__iter__()`, además contiene el método `__next__()` que nos va retornando el siguiente elemento.

In [ ]:
x = [11, 32, 43]
for c in x: 
    print(c)
print(x.__iter__)
next(x) # aqui podemos ver que una lista no es un iterador

Como vemos arriba, las listas no son un iterador, pero podemos obtener un iterador sobre una lista simplemente llamando al método `__iter__()`

In [ ]:
y = iter(x)
print(next(y))
print(next(y))
print(next(y))

Podemos ver la diferencia de iterar sobre una lista que pertenece a un Objeto a iterar sobre el objeto en si mismo. Lo haremos iterando sobre una baraja para imprimir las cartas que contiene.

In [ ]:
class Carta:
    MONOS = {11: 'J', 12: 'Q', 13: 'K'}
    
    def __init__(self, numero, pinta):
        self.pinta = pinta 
        self.numero = numero if numero <=10 else Carta.MONOS[numero]
        
    def __str__(self):
        return "%s %s" % (self.numero, self.pinta)
    def __repr__(self):
        return "%s *%s" % (self.numero, self.pinta)

In [ ]:
    
class Baraja:
    def __init__(self):
        self.cartas = [Carta(n, p) for p in ['Espada', 'Diamante', 'Corazon', 'Trebol'] for n in range(1, 14)]
        
        # La lista por comprension equivale a escribir:
        #self.cartas = []
        #for p in ['Espada', 'Diamante', 'Corazon', 'Trebol']:
        #    for n in range(1, 14):
        #        self.cartas.append(Carta(n, p))

In [ ]:
print([carta for carta in Baraja().cartas])

Si bien la clase Baraja() contiene muchas cartas, no podemos iterar sobre Baraja(), sólo sobre Baraja().cartas (que corresponde a una lista, por lo tanto es iterable). Supongamos que queremos iterar sobre Baraja() directamente, para ello deberíamos agregar el método `__iter__`

In [ ]:
class BarajaIterable():
    def __init__(self):
        self.cartas = [Carta(n, p) for p in ['Espada', 'Diamante', 'Corazon', 'Trebol'] for n in range(1, 14)]
        
    def __iter__(self):
        return iter(self.cartas)
    
    def __len__(self):
        return len(self.cartas)

In [ ]:
print([carta for carta in BarajaIterable().cartas])

Lo que sucede aqui, es que el `for` llama por debajo al metodo `next()`del iterador que es retornado por el metodo `__iter__` de nuestro iterable. Otra forma de hacerlo mas explicito sería:

In [ ]:
baraja = iter(BarajaIterable())
for i in range(len(BarajaIterable())): 
    print(next(baraja))
    if i ==5:
        break

Veamos un ejemplo de cómo crear un iterador (debe contener los métodos `__iter__` y `__next__`)

In [ ]:
class Fib:
    def __init__(self):
        self.prev = 0
        self.actual = 1

    def __iter__(self):
        return self

    def __next__(self):
        valor = self.actual
        self.actual += self.prev
        self.prev = valor
        return valor

In [ ]:
fibonaci = Fib()
N = 15
l = [next(fibonaci) for i in range(N)]
print(l)

## Generadores

Los generadores son un caso especial de los iteradores. Los generadores nos permiten iterar sobre secuencias de datos sin la necesidad de almacenarlos en alguna estructura de datos, evitando el uso innecesario de memoria (usa memoria sólo "on the fly" para cada item mientras se itera). Esto es muy útil cuando queremos realizar cálculos sobre secuencias de números que sólo nos sirven para ese cálculo en particular. La sintaxis para crear generadores es muy parecida a la comprensión de listas, sólo que en vez de paréntesis cuadrados [] usamos paréntesis normales ():

In [ ]:
from sys import getsizeof
g = (b for b in range(100))#por el sólo hecho de usar paréntesis significa que estamos creando un generador
print("Tamaño Generador en bytes:" + str(getsizeof(g)))
l = [b for b in range(100)]#l usa más memoria que g
print("Tamaño lista en bytes:" + str(getsizeof(l)))

Es importante recordar que una vez que terminamos de iterar sobre un generador, el generador desaparece.

In [ ]:
g = (b for b in range(10))
for b in g:
    print(b)

print(next(g))

### Ejemplo: Procesamiento de un archivo

Supongamos este contenido para el archivo logs.txt:

Abr 13, 2021 09:22:34

Jun 14, 2021 08:32:11

May 20, 2021 10:12:54

Dic 21, 2021 11:11:62

WARNING Estamos cercanos a tener un problema.

WARNING Otro warning!

WARNING Casi un bug

WARNING Tenga mucho cuidado

In [ ]:
import sys
inname, outname = "logs.txt", "logs_out.txt"
with open(inname) as infile:
    with open(outname, "w") as outfile:
        warnings = (l.replace('WARNING', '') for l in infile if 'WARNING' in l)
        for l in warnings:
            outfile.write(l)

### Este sería el output:

Estamos cercanos a tener un problema.

Otro warning!

Casi un bug

Tenga mucho cuidado

## Funciones Generadoras

Las funciones en Python también tienen la posibilidad de funcionar como generadores, a través de "yield". El statement "yield" reemplaza a "return", por un lado se encarga de retornar el valor pero además nos asegura que en la próxima llamada a la función, ésta será ejecutada partiendo desde el punto donde quedó en la ejecución anterior, en otras palabras, trabajamos con una función que una vez que "retorna" un valor a través de "yield", está transfierendo el control sólo en forma temporal, asumiendo que pronto será utilizada nuevamente para "generar" más valores. Al llamar a una función generadora se crea un objeto generador, sin embargo, esto no comienza a ejecutar la función. Ejemplo:

In [ ]:
def conteo_dec(n):
    print("Contando en forma decreciente desde {}".format(n))
    while n > 0:
        yield n
        n -= 1

In [ ]:
y = conteo_dec(2)#notar que aquí no se imprime nada

print("{}\n".format(y))#aquí sólo se imprime el objeto

print(str(type(y)) + "\n")# se imprime el tipo de objeto y

print(next(y))
print(next(y))
print(next(y))


In [ ]:
for i in conteo_dec(5):
   print(i)

### También podemos interactuar con la función enviando mensajes

El método `send()` permite enviar un valor hacia el generador, lo que significa que la expresión `yield` lo recibirá. El valor enviado puede ser usado para asignarlo a otra variable, por ejemplo: 

In [ ]:
def grep(pattern): #hace lo mismo que la función grep de la consola de Unix
    print("Buscando a %s" % pattern)
    while True:
        line = (yield)
        if pattern in line:
            print(line)
        
o = grep("Hola")#creo el objeto, no se ejecuta aún la función

next(o)#avanzo al primer yield, aquí se imprime "Buscando a ...."

o.send("Esta linea contiene Hola")
o.send("Esta linea no va a salir")
o.send("Esta linea si va a salir (porque contiene Hola :) )")
o.send("Esta linea tampoco va a salir")
o.send("Hola Ayudantes :D")

## Funciones anónimas

Esa parte fue inspirada por la ayudantia(03) extra del 2019-1

- Python tiene funciones de ***primera clase*** mediante `def`.
Esas son las funciones que hemos ocupado toda vida.

In [ ]:
def sumar(a, b): # Como variable
    return a + b

mi_funcion = sumar
print(mi_funcion(1, 2))

In [ ]:
def iteracion(funcion, lista): # Como parámetro
    aux = 0
    for elemento in lista:
        aux = funcion(aux, elemento)
    return aux

print(iteracion(sumar, [1,2,3]))

### 1. Lambda

- Pero también existen funciones ***anónimas***, definidas mediante `lambda <parámetros>: <retorno>`.

In [ ]:
sumar_anonimo = lambda x, y: x + y # Se utilizan solo donde son creadas

print(sumar_anonimo(1, 2))

- ¿Cuál es la diferencia?

In [ ]:
print(sumar)
print(sumar_anonimo)

Esa funcion se crea localmente, uno puede guardar en una variable, pero esa solo se utiliza donde es creada

### 2. Map

- Recibe **una** función y **al menos** un iterable. Retorna un **generador** con los resultados de aplicar la función a **cada elemento**.

In [ ]:
mi_lista = [] # Queremos sus valores al cuadrado, ¿es esta la forma óptima?

for x in range(100):
    mi_lista.append(x)

In [ ]:
mi_lista = [x for x in range(100)] # Utilizando lista por comprensión 

In [ ]:
def al_cuadrado(iterable): # Version 1.0
    lista_auxiliar = []
    for elemento in iterable:
        lista_auxiliar.append(elemento**2)
    return lista_auxiliar

print(al_cuadrado(mi_lista))

In [ ]:
al_cuadrado_pro = map(lambda x: x**2, mi_lista)

print(al_cuadrado_pro)
print(list(al_cuadrado_pro))

- Para trabajar con **más** de un iterable la función *lambda* deberá recibir el **mismo** número de parámetros que iterables, mientras que map iterará hasta el iterable de **menor** largo.

In [ ]:
llave = ("Mejor ramo", "Mejor serie de super heroes")
valor = ["IIC2233", "The boys", "atributo sobrante que va pasar aqui?"]

personas = map(lambda x, y: (x , y), llave, valor)
print(dict(personas))

### 3. Filter

- Recibe **una** función (que retorne un **boolean**) y **un** iterable. Retorna un **generador** con los **elementos originales** del iterable siempre y cuando al aplicar la función a estos retorne **True**.

In [ ]:
def impares(iterable):
    lista_auxiliar = []
    for elemento in iterable:
        if elemento % 2 != 0:
            lista_auxiliar.append(elemento)
    return lista_auxiliar

print(impares(mi_lista))

In [ ]:
impares_pro = filter(lambda x: x % 2 != 0, mi_lista)

print(list(impares_pro))

### 4. Reduce

- Recibe **una** función (que recibe **dos** parámetros) y **un** iterable. Retorna lo que resulta de aplicar la función `f` al iterable `[s1, s2, s3, ..., sn]` de la siguiente forma: `f(f(f(f(s1, s2), s3), s4), s5), ...`.

![](img\reduce.png)
<!-- ![](img/reduce.png) -->

In [ ]:
mi_frase = ["EL", "que", "usa", "except", "Exception", "as", "error", "no", "es", "digno"]

In [ ]:
def frase(iterable):
    aux = ""
    for elemento in iterable:
        aux = aux + " " + elemento
    return aux

print(frase(mi_frase))

In [ ]:
from functools import reduce
frase_pro = reduce(lambda x, y: x + " " + y, mi_frase)
print(frase_pro)

In [ ]:
al_cuadrado = reduce(lambda x, y: x + y ** 2, [2]) # Ejecuta esta linda funcion
print(al_cuadrado)

- ¿Qué pasó ahí?

In [ ]:
al_cuadrado = reduce(lambda x, y: x + y ** 2, [2], 0)
print(al_cuadrado)

- Si el iterable es de un solo elemento reduce no aplicará la función a menos que exista un ***valor inicial***, que se ingresa como tercer parámetro.

# Ejercicio propuesto: Actividad 11 2019-2
## Parte1: Iterables

Debes completar tres funciones cortas pendientes y que se describen a continuación. El detalle es que Hadani **no permite que utilices los ambientes de iteración for y while** para que aproveches al máximo la magia de los iterables.
La palabra reservada for **solo puede utilizarse dentro del contexto de una estructura por comprensión**,
y **te recomienda el uso de las funciones map, filter y reduce para actuar sobre objetos iterables.**
Para obtener los datos de los PROGRaMONés necesitamos extraer información del archivo de texto
(programon.txt). Para eso necesitamos las funciones:
```python
def obtener_programones(lineas): 
```
Esta función recibe una lista con líneas obtenidas en la función anterior y a partir de los datos obtenidos de las líneas, debe retornar una lista con instancias
de PROGRaMON creadas utilizando la named tuple entregada Programon.

Para consultar la información obtenida anteriormente deberás implementar las siguientes funciones:


In [ ]:
from collections import namedtuple
from functools import reduce

caracteristicas = ['hp', 'ataque', 'defensa', 'velocidad', 'defensa_especial', 'ataque_especial']
Programon = namedtuple('PROGRáMON', ['id', 'nombre', 'tipo_1', 'tipo_2', *caracteristicas])

def obtener_lineas_archivo(ruta): 
    '''A partir de ruta de archivo TXT, retorna iterable con las líneas del archivo.'''
    # Implementado
    with open(ruta, 'rt', encoding='utf-8') as archivo:
        return archivo.readlines()

def obtener_programones(lineas): 
    '''A partir de líneas de texto, retorna un iterable con instancias de Programon.'''
    # Completar
    lineas_separadas = [linea.strip().split(',') for linea in lineas]
    return [Programon(*linea) for linea in lineas_separadas]


In [ ]:
# Cargamos el archivo
lineas = obtener_lineas_archivo('programon.txt')

# Probando obtener_programones
programones = obtener_programones(lineas)
print('Programones cargados:', len(programones))

```python
def obtener_tipos(programones):
```
Esta función recibe una lista con múltiples PROGRaMON
y retorna una lista con los tipos de todos los PROGRaMON presentes en el iterable entregado.
Nota, que cada PROGRaMON  tiene a lo más dos tipos distintos, y el resultado no puede tener
repeticiones. Un ejemplo de esta consulta sería:
```python
obtener_tipos([
PROGRaMON(id='39', nombre='Jigglypuff', tipo_1='Normal', tipo_2='Hada',...),
PROGRaMON(id='183', nombre='Marill', tipo_1='Agua', tipo_2='Hada',...)
])

Y un resultado esperado sería:
['Hada', 'Agua', 'Normal']
```

In [ ]:
def obtener_tipos(programones): 
    '''A partir de un iterable de programones, retorna un iterables de sus tipos sin 
    repeticiones.'''
    # Completar
    pass

In [ ]:
# Probando obtener_tipos
tipos = obtener_tipos(programones)
print('Tipos encontrados:', len(tipos))
print(tipos)

```python
def obtener_programones_de_tipo(programones, tipo): 
```
Esta función recibe una lista con múltiples PROGRaMON  y un *string (str)* que indica un tipo de PROGRaMON  . La función debe
retornar una nueva lista que sólo contenga los PROGRaMON del iterable entregado pero que sean del tipo indicado. Un ejemplo de esta consulta sería:

```python
obtener_programones_de_tipo([
PROGRaMON(id='284', nombre='Masquerain', tipo_1='Bicho', tipo_2='Volador',...),
PROGRaMON(id='301', nombre='Delcatty', tipo_1='Normal', tipo_2='',...),
PROGRaMON(id='347', nombre='Anorith', tipo_1='Roca', tipo_2='Bicho',...)
],
'Bicho'
)
```

In [ ]:
def obtener_programones_de_tipo(programones, tipo):
    '''A partir de un iterable de programones y un tipo, retorna un iterable con aquellos 
    programones del tipo específicado.''' 
    # Completar
    pass

In [ ]:
# Probando obtener_programones_de_tipo
hadas = obtener_programones_de_tipo(programones, 'Hada')
print('Hadas encontradas:', len(hadas))   
print(hadas)

```python
def obtener_cantidad_por_tipo(programones, tipos):
```

Esta función recibe una lista con múltiples PROGRAMON y una lista de strings (str) que indican tipos de PROGRAMON. La función retorna un dict que tiene como llaves los tipos entregados y como valor asociado la cantidad de 2 PROGRAMON de ese tipo que se encuentran en el iterable. Un ejemplo de esta consulta sería:

```python
obtener_cantidad_por_tipo([
PROGRaMON(id='267', nombre='Beautifly', tipo_1='Bicho', tipo_2='Volador',...), PROGRaMON(id='269', nombre='Dustox', tipo_1='Bicho', tipo_2='Veneno',...)
],
  ['Planta', 'Bicho', 'Veneno', 'Volador']
   )
```

In [ ]:
def obtener_cantidad_por_tipo(programones, tipos): 
    '''A partir de iterable de programones e iterable de tipos, retorna diccionario con llaves
    los tipos entregados y cuyos valores corresponden a la cantidad de programones de tal tipo.'''
    # Completar
    pass

In [ ]:
# Probando obtener_cantidad_por_tipo
cantidad_por_tipo = obtener_cantidad_por_tipo(programones, tipos)
print('Diccionario de cantidad por tipo:')
print(cantidad_por_tipo)
print(repr_lista(list(cantidad_por_tipo.items())))